## Unidad 3:

### Cálculo Numérico

- Integración.

**1** Resolver las siguientes integrales con los métodos propuestos. (Simpson, trapecio impropias)

a) $\int_0^1 \frac{x^4(1-x)^4}{1+x^2}\;\mathrm{d}x=\frac{22}{7}-\pi.$

b) $\int_0^\infty \frac{x^3}{e^x-1}\;\mathrm{d}x=\frac{\pi^4}{15}$ (aparece en teoría de Debye para la capacidad calorica en crystales).

c) $\int_0^1 x^{-x}\;\mathrm{d}x=\sum_{n=1}^\infty n^{-n}$ (conocida como "Sophomore's dream").

d) $\int_0^1 [\ln(1/x)]^p\;\mathrm{d}x=p!$ si $0 \le p \le 10$.

e) $\int_0^{2\pi} e^{z\cos\theta}\;\mathrm{d}\theta=2\pi I_0(z)$ (donde $I_0(z)$ es la función de Bessel de primera clase definida en $0 \le z \le 2$).

f) $\int _{0}^{\infty }\cos t^{2}\,dt=\int _{0}^{\infty }\sin t^{2}\,dt={\sqrt {\frac {\pi }{8}}}$.

Compare los resultados de los algoritmos dados en este capítulo a los dados por `scipy` o `sympy`.

b y f opcional, Bonus de 1pt (pista: hacer metodo de gauss-hermite)

**2** La [capacidad calorífica](https://es.wikipedia.org/wiki/Capacidad_calor%C3%ADfica) o capacidad térmica de un cuerpo, $C_V$, se define como el cociente entre la cantidad de energía calorífica transferida a un cuerpo y el cambio de temperatura que experimenta, es decir, $C_V$, mide la energía necesaria para aumentar la temperatura de un cuerpo en una unidad de temperatura. Según la teoría de Debye la capacidad calórica es dada por la integral,

$$
\frac {C_{V}}{Nk}=9\left({T \over T_{\rm {D}}}\right)^{3}\int _{0}^{T_{\rm {D}}/T}{x^{4}e^{x} \over \left(e^{x}-1\right)^{2}}\,dx,
$$

donde $T_D$ es la temperatura de Debye, $k$ es la constate de Bolzmann y $N$ el número de partículas contenidas en el volumen $V$.

a) Use los métodos de integración para graficar la capacidad calórica, $C_V/3Nk$, como función de $T/T_D$ en el intervalo $[0,1.5]$.

b) Einstein había propuesto inicialmente el modelo,

$$\frac{C_V}{3Nk}=\left({\epsilon\over k T}\right)^2{e^{\epsilon/kT}\over \left(e^{\epsilon/kT}-1\right)^2},$$

con energía dada por $\frac{\epsilon}{kT_D} = \frac{\hbar \omega}{kT_D} = \sqrt{3}\frac{\pi}{6}=0.805995977$, donde $\omega$ es definida como la frecuencia del oscilador usado en el modelo de Einstein.
Grafique y compare al modelo de Debye.

**3** **Por librería o elección** En electromagnetismo se puede demostrar que el potencial producido por un anillo de radio $a=1.0$ m en el plano $xy$ con distribución de carga $\lambda$, en un punto P fuera del eje $z$ es dado por,

$$V = \frac{\lambda}{2 \pi \varepsilon_0}\sqrt{\frac{am}{x}} K(m)$$

donde,

$$m = \frac{2}{1 + \frac{z^{2} + x^{2} + a^{2}}{2 a x}},$$

y $K(m)$ es la integral elíptica completa de primera especie definida como,

$$K(m) = \int_{0}^{\pi/2} \frac{d  \phi}{\sqrt{1 - m \text{sen}^{2}(\phi)}}$$

a) Si  $\frac{\lambda}{2\pi\varepsilon_0}=1.0$, ¿cuál es el potencial eléctrico en el punto $(x,y,z)=(1.5,0,1.0)$?<br>

b) Grafique la solución para el caso en que $-100<x<100$ use pasos de $0.01$. ¿Qué puede concluir de la gráfica?si $z=cte$

c) La solución analítica es dada por la serie,

$$ K(m)={\frac {\pi }{2}}\sum _{n=0}^{\infty }\left({\frac {(2n)!}{2^{2n}(n!)^{2}}}\right)^{2}m^{n}={\frac {\pi }{2}}\sum _{n=0}^{\infty }{\bigl (}P_{2n}(0){\bigr )}^{2}m^{n},$$

donde y $P_n$ son los [polinomios de Legendre](#MATERIAL_COMPLEMENTARIO). Equivalentemente, también,

$$K(m)={\frac {\pi }{2}}\left(1+\left({\frac {1}{2}}\right)^{2}m+\left({\frac {1\cdot 3}{2\cdot 4}}\right)^{2}m^{2}+\cdots +\left({\frac {\left(2n-1\right)!!}{\left(2n\right)!!}}\right)^{2}m^{n}+\cdots \right),$$

donde $n!!$ es el doble factorial. Grafique y compare estas tres soluciones a la dada por la integral numérica ¿Qué concluye? (Nota: tenga cuidado con $x\leq 0$, en las series, considere la simetría para esto).
